In [30]:
# pip install openpyxl
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [31]:
# 1. 데이터 불러오기
# pandas로 엑셀 데이터를 불러옵니다.
# file_path는 불러올 엑셀 파일의 경로입니다.
# sheet_name은 불러올 시트의 이름을 지정하는데, 여기서는 '2023년 01월' 데이터를 가져옵니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
file_path = '../data/2023년 01월 서울시 교통량.xlsx'
data = pd.read_excel(file_path, sheet_name="2023년 01월")

# 데이터를 살펴보면 현재 일자와 요일이 일치하지 않습니다. 그렇기 때문에 일자 칼럼을 이용하여 요일을 다시 계산해야 합니다.

In [32]:
# 2. '일자' 컬럼을 datetime 형식으로 변환
# 이를 datetime 형식으로 변환하면 날짜 관련 작업이 더 쉬워집니다.
# pandas의 to_datetime 함수를 사용하여 '일자' 데이터를 날짜 형식으로 변환합니다.
# (참고: '일자' 컬럼은 날짜를 나타내는 값인데, 이 컬럼의 각 값들은 정수 형식(YYYYMMDD)으로 되어 있습니다.)
# 정수 형식에 대한 format을 선언해줘야 합니다.
# 해주지 않으면 1970-01-01 00:00:00.020230101와 같은 형식으로 표현됩니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
data['일자'] = pd.to_datetime(data['일자'], format="%Y%m%d")

In [33]:
# 3. 날짜를 기반으로 요일을 자동으로 계산
# 요일이 정수로 변환되면 학습 모델에서 이를 숫자형 데이터로 처리할 수 있습니다.
# datetime 형식으로 변환된 '일자' 컬럼을 기반으로 요일을 계산합니다.
# dt.weekday는 '월요일'=0, '일요일'=6을 반환하므로, +1을 해주어 '월요일'=1, '일요일'=7로 변환합니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
data['요일'] = data['일자'].dt.weekday + 1

In [34]:
# 4. 데이터 필터링
# '성산로(금화터널)' 지점에서 '유입' 방향으로 들어오는 데이터만을 선택합니다.
# 데이터를 먼저 필터링하여 분석에 필요한 데이터만 남깁니다.
# Boolean indexing을 사용하여 조건에 맞는 데이터만 필터링할 수 있습니다.
# 예: filtered_data = 데이터프레임[데이터프레임['컬럼명'] == '값' & 데이터프레임['컬럼명'] == '값']
# 참고: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing
filtered_data = data[(data['지점명'] == '성산로(금화터널)') & (data['방향'] == '유입')]
print(filtered_data)

           일자  요일        지점명  지점번호  방향            구분     0시     1시     2시  \
0  2023-01-01   7  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  564.0  414.0  268.0   
1  2023-01-02   1  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  294.0  190.0  146.0   
2  2023-01-03   2  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  419.0  271.0  203.0   
3  2023-01-04   3  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  502.0  335.0  247.0   
4  2023-01-05   4  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  523.0  334.0  235.0   
5  2023-01-06   5  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  561.0  403.0  275.0   
6  2023-01-07   6  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  454.0  362.0  256.0   
7  2023-01-08   7  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  434.0  354.0  277.0   
8  2023-01-09   1  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  302.0  238.0  164.0   
9  2023-01-10   2  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  469.0  328.0  223.0   
10 2023-01-11   3  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  525.0  394.0  230.0   
11 2023-01-12   4  성산로(금화터널)  A-01  유입  봉원고가차도->독립문역  510.0  384.0  294.0   

In [41]:
# 5. 피처와 타깃 값 설정
# '요일'을 독립 변수로, '0시' 교통량을 종속 변수로 설정합니다.
# 참고: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
X = filtered_data[['요일']]
y = filtered_data['0시']

In [42]:
# 6. 학습 데이터와 테스트 데이터 분할
# 학습 데이터와 테스트 데이터를 나누는 함수로 train_test_split을 사용합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [43]:
# 7. 선형 회귀 모델 정의 및 학습
# scikit-learn의 LinearRegression 모델을 사용하여 학습 데이터를 학습시킵니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [47]:
# 8. 모델 평가
# 학습된 모델을 평가하기 위해 R² 점수를 사용합니다. 1에 가까울수록 좋은 성능을 의미합니다.
# scikit-learn의 score() 함수는 기본적으로 R² 점수를 반환합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score
r2_score = model.score(X, y)
print(f"R²: {r2_score}")

R²: 0.12665521448009598
